## Data Science Capstone Project
### Segmenting and Clustering Neighborhoods in Toronto
Part 2

## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install geopy
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Prepare Toronto Neighbourhood data

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = dfs[0]

In [3]:
df = df[df.Borough != "Not assigned"]
df.reset_index(drop = True, inplace = True)

for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [4]:
coordinates_df = pd.read_csv('http://cocl.us/Geospatial_data')
df = pd.merge(df, coordinates_df, on="Postal Code")

## Get Toronto map data

In [5]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get Venue data per neighbourhood

In [7]:
CLIENT_ID = 'MZY2AOYQL41BRWYAKPCX25L4B4PWSYGSJGEBRJSPQG01HVL1' # your Foursquare ID
CLIENT_SECRET = '3OFXCQYQPMPWZSRXOZZOV23BJ4UALSPNY5WIAPWSBBEV11SM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MZY2AOYQL41BRWYAKPCX25L4B4PWSYGSJGEBRJSPQG01HVL1
CLIENT_SECRET:3OFXCQYQPMPWZSRXOZZOV23BJ4UALSPNY5WIAPWSBBEV11SM


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [10]:
print(toronto_venues.shape)
toronto_venues.head()

(2140, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [11]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [12]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


## Analyzing Venues per neighbourhood

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [14]:
toronto_onehot.shape

(2140, 272)

In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

### Check what the top 5 Common venues are per neighborhood

In [16]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3  Latin American Restaurant  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.25
1                 Gym  0.12
2      Sandwich Place  0.12
3  Athletics & Sports  0.12
4         Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2       Pharmacy  0.05
3          Diner  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4  Monument / Landmark  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Pizza Place  0.08
2          Restaurant  0.08
3      Sandwich Pla

               venue  freq
0               Park  0.33
1        Swim School  0.33
2           Bus Line  0.33
3      Metro Station  0.00
4  Mobile Phone Shop  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.12
1  Furniture / Home Store  0.06
2                    Bank  0.06
3           Shopping Mall  0.06
4            Burger Joint  0.06


----Little Portugal, Trinity----
                           venue  freq
0                            Bar  0.11
1                     Restaurant  0.05
2          Vietnamese Restaurant  0.05
3  Vegetarian / Vegan Restaurant  0.05
4                    Men's Store  0.05


----Malvern, Rouge----
                        venue  freq
0        Fast Food Restaurant   1.0
1                 Men's Store   0.0
2  Modern European Restaurant   0.0
3           Mobile Phone Shop   0.0
4          Miscellaneous Shop   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                        venue  freq
0              

                       venue  freq
0                     Bakery  0.25
1             Breakfast Spot  0.25
2                 Smoke Shop  0.25
3  Middle Eastern Restaurant  0.25
4                Yoga Studio  0.00


----Willowdale, Willowdale East----
              venue  freq
0  Ramen Restaurant  0.09
1       Pizza Place  0.06
2    Sandwich Place  0.06
3              Café  0.06
4        Restaurant  0.06


----Willowdale, Willowdale West----
            venue  freq
0        Pharmacy  0.14
1   Grocery Store  0.14
2         Butcher  0.14
3  Discount Store  0.14
4     Coffee Shop  0.14


----Woburn----
                venue  freq
0         Coffee Shop  0.50
1   Convenience Store  0.25
2   Korean Restaurant  0.25
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Woodbine Heights----
                venue  freq
0                 Spa  0.12
1        Skating Rink  0.12
2            Bus Stop  0.12
3  Athletics & Sports  0.12
4          Beer Store  0.12


----York Mills West----
        

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Athletics & Sports,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Ice Cream Shop,Pharmacy,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Restaurant


## Cluster the neighborhoods together on common Veneus

In [19]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [21]:
toronto_merged=toronto_merged.dropna()

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Show and define Clusters

### Cluster 1

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North York,0,Intersection,French Restaurant,Coffee Shop,Pizza Place,Hockey Arena
8,East York,0,Pizza Place,Breakfast Spot,Pharmacy,Café,Athletics & Sports
10,North York,0,Bakery,Pub,Pizza Place,Japanese Restaurant,Women's Store
50,North York,0,Gym,Pizza Place,Women's Store,Doner Restaurant,Dessert Shop
63,York,0,Pizza Place,Breakfast Spot,Convenience Store,Caribbean Restaurant,Women's Store
67,Central Toronto,0,Gym / Fitness Center,Food & Drink Shop,Park,Breakfast Spot,Pizza Place
70,Etobicoke,0,Pizza Place,Chinese Restaurant,Intersection,Coffee Shop,Discount Store
72,North York,0,Butcher,Grocery Store,Bank,Coffee Shop,Pharmacy
77,Etobicoke,0,Bus Line,Pizza Place,Sandwich Place,Mobile Phone Shop,Women's Store
82,Scarborough,0,Pizza Place,Fried Chicken Joint,Noodle House,Fast Food Restaurant,Pharmacy


This cluster is harder to define as there are some commen venue's in the borough's but also a lot differences

### Cluster 2

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
57,North York,1,Baseball Field,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner
101,Etobicoke,1,Baseball Field,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner


This Cluster is easy to defined as a area with baseball fileds as those thow borough's have the same common venues

### Cluster 3

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Bakery,Pub,Park,Theater
3,North York,2,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop
4,Downtown Toronto,2,Coffee Shop,Diner,Sushi Restaurant,College Auditorium,Bar
6,Scarborough,2,Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant
7,North York,2,Gym,Beer Store,Japanese Restaurant,Coffee Shop,Restaurant
9,Downtown Toronto,2,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant
12,Scarborough,2,Bar,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner
13,North York,2,Gym,Beer Store,Japanese Restaurant,Coffee Shop,Restaurant
14,East York,2,Park,Curling Ice,Spa,Beer Store,Bus Stop
15,Downtown Toronto,2,Café,Coffee Shop,Gastropub,American Restaurant,Restaurant


This cluster is also harder to define because it has a lot of different type of venues.

### Cluster 4

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,3,Park,Pool,Food & Drink Shop,Dessert Shop,Dim Sum Restaurant
32,Scarborough,3,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner
35,East York,3,Park,Convenience Store,Falafel Restaurant,Event Space,Ethiopian Restaurant
61,Central Toronto,3,Park,Swim School,Bus Line,Dog Run,Dim Sum Restaurant
64,York,3,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
66,North York,3,Park,Convenience Store,Falafel Restaurant,Event Space,Ethiopian Restaurant
68,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,Dim Sum Restaurant
85,Scarborough,3,Park,Playground,Sculpture Garden,Dog Run,Dessert Shop
91,Downtown Toronto,3,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant


This cluster is easely definable by its parks and playgrounds.

### Cluster 5

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
83,Central Toronto,4,Lawyer,Women's Store,Donut Shop,Dim Sum Restaurant,Diner


As this cluster has only one Neigbourhood it can be easely defined as the neigbhourhoood with the lawyer offices